In [14]:
import tensorflow as tf
import numpy as np

def load_tfrecord_data(filename):
    dataset = tf.data.TFRecordDataset(filename)
    def parse_record(record):
        feature_description = {
            'spec': tf.io.FixedLenFeature([99, 43, 1], tf.float32),
            'label': tf.io.FixedLenFeature([3], tf.float32)
        }
        example = tf.io.parse_single_example(record, feature_description)
        return example['spec'], example['label']

    dataset = dataset.map(parse_record).prefetch(tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=False)
    num_samples = 535287 #sum(1 for _ in dataset)
    
    val_samples = int(np.floor(0.1 * num_samples))
    test_samples = int(np.floor(0.1 * num_samples))
    train_samples = num_samples - val_samples - test_samples

    train_dataset = dataset.take(train_samples)
    val_dataset = dataset.skip(train_samples).take(val_samples)
    test_dataset = dataset.skip(train_samples + val_samples).take(test_samples)

    return train_dataset, val_dataset, test_dataset

train_dataset, _, _ = load_tfrecord_data("specs.tfrecord")

converter = tf.lite.TFLiteConverter.from_saved_model("trained.model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for data, _ in train_dataset.take(300):
        yield [tf.dtypes.cast(tf.expand_dims(data, 0), tf.float32)]
#     for d, l in train_dataset:
#         # Get sample input data as a numpy array in a method of your choosing.
#         yield [d, l]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

2023-04-24 12:27:52.487624: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-04-24 12:27:52.487738: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


56296

In [15]:
interpreter = tf.lite.Interpreter(model_path=str("converted_model.tflite"))
interpreter.allocate_tensors()

# complete_train_Y = np.concatenate((training_spectrogram['Y'], validation_spectrogram['Y'], test_spectrogram['Y']))
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]


In [16]:
good  = 0
no_samples = 1000
for data, label in train_dataset.take(no_samples):
    data = tf.expand_dims(data, 0)
    
    interpreter.set_tensor(input_details["index"], data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    
    good  = good + 1 if int(tf.argmax(label, axis=0)) == int(tf.argmax(output, axis=0)) else good

print(good/no_samples)

0.996
